In [1]:
from moist_konrad import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
###PARAMETERS OF THE TEST:

#coupled surface-atmosphere: 
#if true, surface fluxes and surface temperature change at each step, constant otherwise
coupled = True

SH_f = 15.
LH_f = 140.
Ts_f = np.array([287.])

RLS = 1/1

#Strong coupling surface-atmosphere:
#If true the atmosphere moist adiabat starts at surface temperature
#If false find the atmopsheric moist adiabat that conserves energy
strong = False

popke = False
if popke == True:
    solar = 510.
    angle = 47.9
else:
    solar = 480.
    angle = 42.05

A = 0.2

In [3]:
### INITIALIZATION
E_imbalance = 0.

plev, phlev = konrad.utils.get_pressure_grids(1000e2, 100, 128)
atmosphere = konrad.atmosphere.Atmosphere(phlev)
clearsky = konrad.cloud.ClearSky.from_atmosphere(atmosphere)
surface = konrad.surface.SlabOcean(
    temperature=288,
    heat_sink=0,
    depth = 50,
    albedo = 0.5)

radiation = konrad.radiation.RRTMG(solar_constant=solar, zenith_angle = angle)

ndays = 10000
timestep = 0.5 #days
nsteps = int(ndays/timestep)

printcounter = 0
status = 0

tol = 1e-5

dE_1 = [1e8]
OLR_1 = [1e8]
NSW_1 = [1e8]
T_sfc_1 = [1e8]
toa_1 = [1e8]
T_low_1 = [1e8]
atm_rad_1 = [1e8]
sfc_rad_1 = [1e8]
SH_1 = [1e8]
LH_1 = [1e8]
PH_1 = [1e8]

In [ ]:
%%time
### RCPE COMPUTATION

surface['temperature'] = Ts_f
surface.albedo = A
print(surface['temperature'],surface.albedo)

conv_top = coldpoint(atmosphere['T'][0])
RH = manabe_rh(0.77,atmosphere['plev'])
atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],conv_top)
T_atm_low = surface['temperature']

print(surface['temperature'],surface.albedo)

#### RCE FOR PRESENT-DAY CO2
atmosphere['CO2'][0] = 348/1e6


for i in range(nsteps):
    #surface fluxes
    z = height(atmosphere['plev'],atmosphere['T'][0],
               atmosphere['phlev'][0],surface['temperature'])
    zini = 0
    
    T_ini,vmr_ini = T_vmr_z(T_atm_low,atmosphere['T'][0][0].copy(),
                            atmosphere['H2O'][0,0].copy(),atmosphere['H2O'][0,1].copy(),
                            z,zini)
    
    #compute the fluxes or input the forcing
    if coupled == True:
        LH = latent_heat(vmr_ini, T_ini,atmosphere['phlev'][0],RLS)
        SH = sensible_heat(T_ini,surface['temperature'],atmosphere['phlev'][0],RLS)
        
    else:
        LH = LH_f
        SH = SH_f
        
    
    #convective adjustment of the armosphere
    atmosphere,surface,radiation,net_rad_surface,atm_rad,\
    T_atm_low,E_imbalance,prec_mass,RH,cold_point = RCPE_step_DSE(timestep,
                                                                          atmosphere,surface,radiation,clearsky,
                                                                          SH,LH,surface.albedo,T_atm_low,
                                                                      strong_coupling = strong,
                                                                      constrain_RH = True)
    
        
    
    #update surface temperature with energetic fluxes (latent and radiative)
    if coupled == True:
        surface['temperature'] = surface['temperature'] + ((net_rad_surface - SH - LH)
                               * seconds_day * timestep / surface.heat_capacity)
    
    else:
        surface['temperature'] = Ts_f
    
    
    
    status += 1
    if printcounter == 200:
        print(status/2)
        printcounter = 0
        Flux = np.maximum(net_rad_surface,np.maximum(abs(atm_rad),SH+LH))
        Flux = net_rad_surface
        Flux = (abs(atm_rad)+SH+LH+net_rad_surface)/3
        prec_eff = np.maximum(0.,np.minimum(1.,LH/Flux))
        prec_eff = LH/Flux
        print(surface['temperature'],SH,LH,prec_mass*Lv/seconds_day/timestep,
              E_imbalance,prec_eff)
        print(RH[0])
        print(T_atm_low)
    printcounter += 1

    
    dE_1.append(E_imbalance.copy())
    OLR_1.append(radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy())
    NSW_1.append(radiation['sw_flxd'][0,-1].copy() - radiation['sw_flxu'][0,-1].copy())
    T_sfc_1.append(surface['temperature'])
    toa_1.append(radiation['toa'].copy())
    T_low_1.append(T_ini.copy())
    atm_rad_1.append(atm_rad.copy())
    sfc_rad_1.append(net_rad_surface.copy())
    SH_1.append(SH)
    LH_1.append(LH)
    PH_1.append(prec_mass.copy()*Lv/seconds_day/timestep)
    
    if abs(T_sfc_1[i]-T_sfc_1[i-1])<tol and abs(toa_1[i]-toa_1[i-1])<tol:
        print(i)
        break
    
    
### Record RCE 1xCO2
RH_1 = RH.copy()
Mw_1 = column_water_mass(atmosphere['H2O'][0],atmosphere['phlev']).copy()
vmr_1 = atmosphere['H2O'][0].copy()
T_atm_1 = atmosphere['T'][0].copy()
Prec_1 = prec_mass.copy()/timestep

[287.] 0.2
[287.] 0.2
[1.42441667e+08] [-4827523.75390577]
[1.42431147e+08] [-5993467.19103622]
[1.42547319e+08] [-5811212.82146358]
[1.42838726e+08] [-5706635.09693766]
[1.43255832e+08] [-5643030.47271347]
[1.43758896e+08] [-5607500.60385799]
[1.44320293e+08] [-5589132.37665939]
[1.44917892e+08] [-5581689.72790909]
[1.45536523e+08] [-5582245.90885305]
[1.46164118e+08] [-5588932.39487219]
[1.46794813e+08] [-5597116.75745916]
[1.47419994e+08] [-5610146.65245199]
[1.48044282e+08] [-5616658.32884312]
[1.48676031e+08] [-5613058.880867]
[1.49318279e+08] [-5604787.05915785]
[1.49972794e+08] [-5594800.05217266]
[1.50642891e+08] [-5583135.46031809]
[1.51343526e+08] [-5569502.37013197]
[1.52062895e+08] [-5551640.27747869]
[1.52802502e+08] [-5531836.73284197]
[1.53563696e+08] [-5510101.6999321]
[1.54348247e+08] [-5486334.08740711]
[1.55158323e+08] [-5460510.94426489]
[1.55995427e+08] [-5432484.79642534]
[1.56861452e+08] [-5402136.99184752]
[1.5775894e+08] [-5369353.86255121]
[1.58689058e+08] [-5

[1.7863164e+08] [-6433516.47644997]
[1.78640619e+08] [-6433950.01074219]
[1.78650032e+08] [-6434646.25281954]
[1.78659858e+08] [-6435134.18963957]
[1.78670041e+08] [-6435643.24480963]
[1.78680561e+08] [-6436113.28319263]
[1.78691391e+08] [-6436843.62726545]
[1.78702518e+08] [-6437357.12379789]
[1.78713906e+08] [-6437965.66422987]
[1.78725554e+08] [-6438624.40039825]
[1.78737425e+08] [-6439208.35829258]
[1.78749492e+08] [-6439725.45538664]
[1.78761734e+08] [-6440617.59229231]
[1.78774168e+08] [-6441226.01630688]
[1.78786758e+08] [-6441817.24774313]
[1.7879948e+08] [-6442371.91305399]
[1.78812327e+08] [-6443107.60515642]
[1.78825299e+08] [-6444017.50063658]
[1.78838394e+08] [-6444556.50648594]
[1.78851576e+08] [-6445361.17020321]
[1.78864856e+08] [-6446010.09365654]
[1.78878209e+08] [-6446814.79195261]
[1.78891646e+08] [-6447636.30153418]
[1.78905151e+08] [-6448178.41644573]
[1.78918696e+08] [-6449124.7451725]
[1.78932303e+08] [-6449804.87653828]
[1.78947109e+08] [-6450449.74467993]
[1.7

[1.81318114e+08] [-6594964.50285959]
[1.81328749e+08] [-6595811.57414341]
[1.81339375e+08] [-6596670.16558361]
[1.81349991e+08] [-6597429.01359653]
[1.81360583e+08] [-6598123.53817654]
[1.81371152e+08] [-6598834.09158945]
[1.81381706e+08] [-6599516.97165728]
[1.81392237e+08] [-6600111.05860329]
[1.81402751e+08] [-6600847.77474451]
[1.81413258e+08] [-6601505.17490816]
[1.81423749e+08] [-6602157.7480979]
[1.81434225e+08] [-6602629.00329494]
[1.81444674e+08] [-6603232.90400267]
[1.81455106e+08] [-6603914.20244789]
[1.81465537e+08] [-6604947.36023235]
[1.81475982e+08] [-6605779.64124298]
[1.81486414e+08] [-6606467.30140638]
[1.81496826e+08] [-6607271.12908602]
[1.81507233e+08] [-6607887.33253717]
[1.81517613e+08] [-6608591.42508173]
[1.81527982e+08] [-6609390.67179441]
[1.81538352e+08] [-6610084.24377108]
[1.81548708e+08] [-6610692.22423315]
[1.81559039e+08] [-6611363.56286335]
[1.81569357e+08] [-6611934.57957649]
[1.81579659e+08] [-6612687.81252718]
[1.8158995e+08] [-6613411.68833637]
[1.

[1.83309582e+08] [-6745749.38222599]
[1.83318449e+08] [-6746464.22228336]
[1.8332731e+08] [-6747237.47848701]
[1.83336163e+08] [-6747741.20591259]
[1.83344997e+08] [-6748551.2972827]
[1.83353829e+08] [-6749127.57420111]
[1.83362646e+08] [-6749930.00739813]
[1.8337148e+08] [-6750567.58941793]
[1.8338029e+08] [-6751093.61044836]
[1.8338908e+08] [-6751649.87035704]
[1.83397854e+08] [-6752533.22014666]
[1.83406637e+08] [-6753116.75365877]
[1.83415398e+08] [-6753887.16414833]
[1.83424165e+08] [-6754733.73883533]
[1.83432924e+08] [-6755484.36684608]
[1.83441676e+08] [-6756098.59683704]
[1.83450409e+08] [-6756776.47797489]
[1.83459141e+08] [-6757441.87179184]
[1.83467869e+08] [-6758159.15900278]
[1.83476591e+08] [-6758785.18515968]
[1.83485301e+08] [-6759323.24933481]
[1.83493988e+08] [-6759813.7674098]
[1.83502665e+08] [-6760511.30096531]
[1.83511345e+08] [-6761219.09115696]
[1.83520019e+08] [-6761910.02751255]
[1.83528702e+08] [-6762555.15185976]
[1.83537376e+08] [-6763327.21622944]
[1.8354

[1.85120863e+08] [-6891651.77285147]
[1.85128489e+08] [-6892366.72834587]
[1.85136119e+08] [-6893018.81648636]
[1.85143742e+08] [-6893850.88054562]
[1.85151381e+08] [-6894618.62434387]
[1.85159013e+08] [-6895080.95528936]
[1.85166613e+08] [-6895548.75908327]
[1.85174195e+08] [-6896195.76808119]
[1.8518178e+08] [-6896986.96439075]
[1.85189083e+08] [-6897482.86391354]
[1.85196659e+08] [-6898160.12991571]
[1.85204238e+08] [-6898795.79768229]
[1.85211816e+08] [-6899437.87282705]
[1.85219385e+08] [-6900175.9236331]
[1.85226959e+08] [-6900836.58941174]
[1.85234527e+08] [-6901866.36064005]
[1.85242122e+08] [-6902429.67602205]
[1.85249693e+08] [-6903000.83534813]
[1.85257246e+08] [-6903699.75152397]
[1.85264793e+08] [-6904114.54750681]
[1.85272316e+08] [-6904604.31112051]
[1.85279828e+08] [-6905319.45253038]
[1.85287351e+08] [-6905859.39081621]
[1.85294863e+08] [-6906352.73234606]
[1.85302362e+08] [-6906888.86163759]
[1.85309859e+08] [-6907561.93743229]
[1.85317363e+08] [-6908459.18996334]
[1.

...

[1.98207318e+08] [-8770246.25429535]
[1.9820735e+08] [-8770244.92970371]
[1.98207381e+08] [-8770252.70417166]
[1.98207412e+08] [-8770258.91742468]
[1.98207443e+08] [-8770263.04534626]
[1.98207474e+08] [-8770294.33888054]
[1.98207507e+08] [-8770377.43843746]
[1.98207543e+08] [-8770388.82756186]
[1.98207577e+08] [-8770390.54992151]
[1.98207609e+08] [-8770397.60259724]
[1.9820764e+08] [-8770402.15830374]
[1.98207671e+08] [-8770402.15481472]
[1.98207701e+08] [-8770400.35416794]
[1.98207731e+08] [-8770447.7049737]
[1.98207764e+08] [-8770454.3767128]
[1.98207796e+08] [-8770458.7507658]
[1.98207828e+08] [-8770500.97786236]
[1.98207861e+08] [-8770500.0046649]
[1.98207894e+08] [-8770505.53215742]
[1.98207926e+08] [-8770512.76386166]
[1.98207957e+08] [-8770513.82273722]
[1.98207989e+08] [-8770625.06627846]
[1.98208028e+08] [-8770631.98277426]
[1.98208064e+08] [-8770638.46714163]
[1.98208098e+08] [-8770645.76469803]
[1.98208131e+08] [-8770646.50687075]
[1.98208163e+08] [-8770648.01678038]
[1.9820

[1.98212436e+08] [-8772179.59932089]
[1.98212453e+08] [-8772185.71490192]
[1.9821247e+08] [-8772192.30689859]
[1.98212488e+08] [-8772198.3815484]
[1.98212505e+08] [-8772213.32574224]
[1.98212523e+08] [-8772217.97518206]
[1.9821254e+08] [-8772220.99712706]
[1.98212556e+08] [-8772227.79689217]
[1.98212572e+08] [-8772233.85071468]
[1.98212589e+08] [-8772240.02682972]
[1.98212605e+08] [-8772244.30971146]
[1.98212621e+08] [-8772248.3358283]
[1.98212637e+08] [-8772252.59021807]
[1.98212653e+08] [-8772258.02630615]
[1.98212669e+08] [-8772263.63313246]
[1.98212685e+08] [-8772264.81227112]
[1.98212702e+08] [-8772269.32459402]
[1.98212718e+08] [-8772274.18853331]
[1.98212734e+08] [-8772278.92399263]
[1.9821275e+08] [-8772326.47589493]
[1.98212772e+08] [-8772333.70461082]
[1.98212793e+08] [-8772339.65898132]
[1.98212809e+08] [-8772345.89594126]
[1.98212825e+08] [-8772343.19237709]
[1.98212841e+08] [-8772349.30888653]
[1.98212856e+08] [-8772354.31226301]
[1.98212871e+08] [-8772358.66887426]
[1.982

[1.98217116e+08] [-8773734.85525465]
[1.98217139e+08] [-8773740.87478828]
[1.98217163e+08] [-8773739.66148329]
[1.98217184e+08] [-8773745.50916719]
[1.98217207e+08] [-8773742.08339739]
[1.98217229e+08] [-8773746.44501495]
[1.9821725e+08] [-8773746.27445555]
6400.5
[297.83980485] [0.01218611] [160.14947531] [160.1431388] [-0.00081418] [0.99970791]
0.8474309543337648
[297.83928644]
[1.98217273e+08] [-8773743.68311977]
[1.98217296e+08] [-8773734.49523735]
[1.98217317e+08] [-8773737.39648724]
[1.9821734e+08] [-8773742.24974823]
[1.98217363e+08] [-8773751.14748573]
[1.98217385e+08] [-8773755.85179329]
[1.98217408e+08] [-8773761.48494005]
[1.98217431e+08] [-8773764.44070911]
[1.98217452e+08] [-8773768.11368084]
[1.98217475e+08] [-8773767.64206076]
[1.98217496e+08] [-8773788.17987108]
[1.9821752e+08] [-8773793.17795181]
[1.98217544e+08] [-8773807.17811441]
[1.98217567e+08] [-8773840.3614397]
[1.98217593e+08] [-8773837.61099577]
[1.98217616e+08] [-8773839.36887598]
[1.9821764e+08] [-8773844.35

[1.98220928e+08] [-8774998.1526885]
[1.98220946e+08] [-8774998.55036736]
[1.98220962e+08] [-8775000.70039225]
[1.98220978e+08] [-8775003.7945838]
[1.98220993e+08] [-8775008.22893238]
[1.98221007e+08] [-8775021.47955084]
[1.98221022e+08] [-8775026.16666508]
[1.98221037e+08] [-8775030.92128468]
[1.98221051e+08] [-8775034.08542109]
[1.98221065e+08] [-8775039.18970633]
[1.98221079e+08] [-8775042.76559401]
[1.98221093e+08] [-8775047.98600626]
[1.98221106e+08] [-8775049.29401922]
[1.9822112e+08] [-8775045.47841835]
[1.98221133e+08] [-8775041.12474108]
[1.98221145e+08] [-8775046.00640345]
[1.98221159e+08] [-8775047.0600853]
[1.9822117e+08] [-8775040.98850584]
[1.98221181e+08] [-8775040.77095222]
[1.98221193e+08] [-8775067.50008821]
[1.98221205e+08] [-8775073.13581133]
[1.98221217e+08] [-8775078.16228819]
[1.98221229e+08] [-8775077.79978943]
[1.9822124e+08] [-8775164.89773893]
[1.98221257e+08] [-8775168.36615658]
[1.98221272e+08] [-8775282.13721371]
[1.98221293e+08] [-8775288.26789284]
[1.9822

[1.98223918e+08] [-8776642.90663481]
[1.98223936e+08] [-8776642.7534914]
[1.98223951e+08] [-8776647.96964121]
[1.98223964e+08] [-8776653.14992094]
[1.98223977e+08] [-8776656.54026842]
[1.98223988e+08] [-8776655.26674604]
[1.98223999e+08] [-8776658.64467859]
[1.98224011e+08] [-8776662.67184782]
[1.98224021e+08] [-8776697.25308418]
[1.98224035e+08] [-8776696.38060951]
[1.98224046e+08] [-8776701.58931732]
[1.9822406e+08] [-8776705.52152491]
[1.9822407e+08] [-8776714.20142651]
[1.98224083e+08] [-8776717.81485319]
[1.98224094e+08] [-8776722.36196232]
[1.98224106e+08] [-8776726.96965885]
[1.98224116e+08] [-8776729.02702188]
[1.98224128e+08] [-8776728.59271431]
[1.98224137e+08] [-8776746.56299543]
[1.9822415e+08] [-8776749.83127689]
[1.98224161e+08] [-8776749.33213377]
[1.98224173e+08] [-8776753.46418285]
[1.98224183e+08] [-8776759.8586359]
[1.98224196e+08] [-8776760.58388567]
[1.98224206e+08] [-8776766.10427952]
[1.98224219e+08] [-8776766.70722961]
[1.98224231e+08] [-8776772.15766191]
[1.982

In [ ]:
print('P',Prec_1)
print('LH',LH_1[-1])
print('SH',SH_1[-1])
print('R_sfc',sfc_rad_1[-1])
print('R_sfc-LH-SH',sfc_rad_1[-1]-SH_1[-1]-LH_1[-1])
print('R_atm', atm_rad_1[-1])
print('R_atm+SH+LP', atm_rad_1[-1] + SH_1[-1] + Lv*Prec_1/seconds_day)
print('toa indirect',sfc_rad_1[-1]+atm_rad_1[-1])
print('toa direct',toa_1[-1])
print('Energy imbalance',dE_1[-1])
print('Energy imbalance in mm/day', dE_1[-1]/Lv *seconds_day)
print('E',LH_1[-1]/Lv * seconds_day)
print('RH_surface',RH_1[0])
print('T_sfc',T_sfc_1[-1])
print('T_atm low',T_low_1[-1])
print('WVP',Mw_1)

In [ ]:
fig, ax = plt.subplots(figsize=(3*6.4, 3*4.8))
SH = np.array(SH_1[1:])
PH = np.array(PH_1[1:])
LH = np.array(LH_1[1:])
Fsfc = np.zeros((len(LH)))
for i in range(len(LH)):
    Fsfc[i] = SH[i] + LH[i]
Rsfc = np.array(sfc_rad_1[1:])
Ratm = np.array(atm_rad_1[1:])

#plt.plot(SH[:],label='SH')
plt.plot(LH[:],label='LH')
plt.plot(PH[:],label='PH')
plt.plot(Fsfc[:],label='Fsfc')
plt.plot(Rsfc[:],label='Rsfc')
plt.plot(-Ratm[:],label='Ratm')
plt.ylim(None,200)
plt.legend()
plt.show()

plt.plot(LH[:],label='LH')
plt.plot(PH[:],label='PH')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(1.5*6.4, 1.5*4.8))
T_sfc_rec = np.array(T_sfc_1[1:])
T_low_rec = np.array(T_low_1[1:])
plt.plot(T_sfc_rec)
plt.plot(T_low_rec)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(1.5*4.8, 1.5*6.4))


ax.plot(T_atm_1, atmosphere['plev']/100,
               alpha = 1, color = 'royalblue', lw = 1.5, label = 'CO$_2$ x 1')
ax.scatter(T_sfc_1[-1], atmosphere['phlev'][0]/100,
               alpha = 1, color = 'royalblue', s = 30, marker ='x')

ax.set_yscale('log')
ax.yaxis.set_inverted(True)
ax.set_xlabel('Temperature (K)')
ax.set_ylabel('Pressure (hPa)')

plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(1.5*4.8, 1.5*6.4))

bot = 0
top = 4

ax.plot(T_atm_1[bot:top], atmosphere['plev'][bot:top]/100,
               alpha = 1, color = 'royalblue', lw = 1.5, label = 'CO$_2$ x 1')
ax.scatter(T_sfc_1[-1], atmosphere['phlev'][0]/100,
               alpha = 1, color = 'royalblue', s = 30, marker ='x')

ax.set_yscale('log')
ax.yaxis.set_inverted(True)
ax.set_xlabel('Temperature (K)')
ax.set_ylabel('Pressure (hPa)')

plt.legend()
plt.show()